In [1]:
import os
os.chdir(r"C:\Users\GilseungAhn\Google 드라이브\작업\온라인 강의 머신러닝 성능 향상을 위한 데이터 탐색과 전처리 심화\데이터\실습 데이터 - #2")

import pandas as pd
df = pd.read_csv("Instacart Market Basket Analysis.csv")

In [2]:
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [3]:
df['product_id'].value_counts(normalize = True).head(100)

24852    0.013524
13176    0.011180
21137    0.007868
21903    0.007066
47626    0.005875
47766    0.005351
47209    0.005267
16797    0.004690
26209    0.004357
27966    0.004005
39275    0.003587
27845    0.003545
30391    0.003332
45007    0.003314
22935    0.003098
24964    0.003003
4920     0.002931
46979    0.002794
40706    0.002761
8518     0.002757
4605     0.002717
42265    0.002598
45066    0.002565
31717    0.002547
5876     0.002531
44632    0.002426
43352    0.002368
28204    0.002352
5450     0.002241
8424     0.002118
           ...   
22825    0.001367
39928    0.001358
15290    0.001346
33198    0.001302
27344    0.001291
18465    0.001276
27521    0.001272
16759    0.001258
33731    0.001223
9839     0.001194
38739    0.001179
27156    0.001138
23909    0.001112
5785     0.001109
46906    0.001103
196      0.001088
11777    0.001078
28199    0.001067
33000    0.001066
44142    0.001060
47144    0.001058
40604    0.001046
5025     0.001044
42736    0.001036
21709    0

In [4]:
product_list_per_order = df.groupby('order_id')['product_id'].apply(list)

In [5]:
# 구매 기록 데이터 -> One hot encoding 데이터
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder() # 인스턴스화
one_hot_df = encoder.fit(product_list_per_order).transform(product_list_per_order) # 결과: ndarray
one_hot_df = pd.DataFrame(one_hot_df, columns = encoder.columns_)
one_hot_df.head() # 매우 희소 (sparse)

,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
from mlxtend.frequent_patterns import *
frequent_item_df = apriori(one_hot_df, min_support = 0.003) # 0.3% 이상 구매한 상품만 대상으로 함
result = association_rules(frequent_item_df, metric = 'confidence', min_threshold = 0.1)
result[['antecedents', 'consequents', 'support', 'confidence']].sort_values(by = 'confidence', ascending = False).to_csv("연관규칙탐사결과.csv", index = False)

In [9]:
result.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(12022),(1815),0.014206,0.008887,0.003018,0.212446,23.906409,0.002892,1.258471
1,(1815),(12022),0.008887,0.014206,0.003018,0.339623,23.906409,0.002892,1.492773
2,(3640),(17300),0.028672,0.074568,0.004146,0.144604,1.939221,0.002008,1.081875
3,(3640),(19574),0.028672,0.142719,0.008163,0.284689,1.994754,0.004071,1.198473
4,(3640),(20648),0.028672,0.045980,0.003780,0.131845,2.867432,0.002462,1.098905
